In [1]:
import pystac
# import pypgstac
# import psycopg
from datetime import datetime

In [2]:
store = "https://data.eodc.eu/collections/CLMS/CLMS.zarr"

extent = pystac.Extent(
    spatial=pystac.SpatialExtent([[8.45, 45.60, 18.07, 49.83]]),
    temporal=pystac.TemporalExtent([[datetime(2019, 1, 1), datetime(2022, 12, 31)]])
)

collection = pystac.Collection(
    id="clms",
    title="Copernicus Land Monitoring Service",
    description="The Copernicus Land Monitoring Service (CLMS) is a component of the European Union’s Copernicus Earth observation programme that provides systematic information on land cover, land use, and biophysical parameters across Europe and globally. It delivers consistent, long-term datasets derived primarily from satellite imagery to support environmental monitoring, spatial planning, agriculture, forestry, and climate applications. The service produces several product lines, including pan-European, local, and global components, enabling both high-resolution mapping and large-scale assessments.",
    extent=extent)

collection.stac_extensions = [
    "https://stac-extensions.github.io/zarr/v1.1.0/schema.json",
]

collection.license = "CC-BY-4.0"


collection.links = [l for l in collection.links if l.rel != "item"]


In [3]:
thumbnail = pystac.Asset(
    href="https://raw.githubusercontent.com/koenifra/CLMS_zarr/main/CLMS_thumbnail.png",
    media_type="image/png",
    roles=["thumbnail"],
    title="CLMS thumbnail"
)

collection.add_asset("thumbnail", thumbnail)


In [4]:
bbox = [8.45, 45.60, 18.07, 49.83]
geometry = {
    "type": "Polygon",
    "coordinates": [[
        [bbox[0], bbox[1]],
        [bbox[2], bbox[1]],
        [bbox[2], bbox[3]],
        [bbox[0], bbox[3]],
        [bbox[0], bbox[1]],
    ]]
}

In [5]:
item_vpp = pystac.Item(
    id="clms-vpp",
    geometry=geometry,
    bbox=bbox,
    datetime=None,  # because it's a time range
    properties={
        "title": "Vegetation Phenology and Productivity",
        "description": "HR-VPP product suite provides information on vegetation status and dynamics.",
        "start_datetime": datetime(2019, 1, 1).isoformat() + "Z",
        "end_datetime": datetime(2022, 12, 31).isoformat() + "Z",
    },
)

In [6]:
asset_vpp = pystac.Asset(
    href=store,
    media_type="application/vnd.zarr; version=3",
    roles=["data"],
    title="Vegetation Phenology and Productivity",
    description="HR-VPP product suite provides information on vegetation status and dynamics.",
)

asset_vpp.extra_fields.update({
    # "zarr:zarr_format": 3,
    # "zarr:node_type": "group",
    # "zarr:group": "VPP",

    "xarray:open_zarr_kwargs": {
        "group": "VPP",
        "zarr_version": 3,
        "consolidated": True,
    },

    "bands": [
            {"name": "AMPL", "description": "Season Amplitude"},
            {"name": "EOSD", "description": "End-of-season Date"},
            {"name": "EOSV", "description": "End-of-season Value"},
            {"name": "LENGTH", "description": "Season Length"},
            {"name": "LSLOPE", "description": "Slope of the Green-up Period"},
            {"name": "MAXV", "description": "Season Maximum Value"},
            {"name": "MINV", "description": "Season Minimum Value"},
            {"name": "QFLAG", "description": "Quality Flag"},
            {"name": "RSLOPE", "description": "Slope of the Green-down Period"},
            {"name": "SOSD", "description": "Start-of-season Date"},
            {"name": "SOSV", "description": "Start-of-season Value"},
            {"name": "SPROD", "description": "Seasonal Productivity"},
            {"name": "TPROD", "description": "Total Productivity"},
        ],
})

item_vpp.add_asset("data", asset_vpp)


In [7]:
item_st = pystac.Item(
    id="clms-st",
    geometry=geometry,
    bbox=bbox,
    datetime=None,
    properties={
         "title": "Seasonal Trajectories",
        "description": "ST product is a filtered time series of PPI provided yearly on a 10-daily basis.",
        "start_datetime": datetime(2019, 1, 1).isoformat() + "Z",
        "end_datetime": datetime(2022, 12, 31).isoformat() + "Z",
    },
)

In [8]:
asset_st = pystac.Asset(
    href=store,
    media_type="application/vnd.zarr; version=3",
    roles=["data"],
    title="Seasonal Trajectories",
    description="ST product is a filtered time series of PPI provided yearly on a 10-daily basis.",
)
asset_st.extra_fields.update({
    # "zarr:zarr_format": 3,
    # "zarr:node_type": "group",
    # "zarr:group": "ST",

    "xarray:open_zarr_kwargs": {
        "group": "ST",
        "zarr_version": 3,
        "consolidated": True,
    },

    "bands": [
        {"name": "PPI", "description": "Plant Phenology Index"},
        {"name": "QFLAG", "description": "Quality Flag"},
    ],
})

item_st.add_asset("data", asset_st)

In [9]:
# collection.add_item(item_vpp)
# collection.add_item(item_st)

In [10]:
collection

<Collection id=clms>

In [11]:
print(collection.to_dict())

{'type': 'Collection', 'id': 'clms', 'stac_version': '1.1.0', 'description': 'The Copernicus Land Monitoring Service (CLMS) is a component of the European Union’s Copernicus Earth observation programme that provides systematic information on land cover, land use, and biophysical parameters across Europe and globally. It delivers consistent, long-term datasets derived primarily from satellite imagery to support environmental monitoring, spatial planning, agriculture, forestry, and climate applications. The service produces several product lines, including pan-European, local, and global components, enabling both high-resolution mapping and large-scale assessments.', 'links': [], 'stac_extensions': ['https://stac-extensions.github.io/zarr/v1.1.0/schema.json'], 'title': 'Copernicus Land Monitoring Service', 'extent': {'spatial': {'bbox': [[8.45, 45.6, 18.07, 49.83]]}, 'temporal': {'interval': [['2019-01-01T00:00:00Z', '2022-12-31T00:00:00Z']]}}, 'license': 'CC-BY-4.0', 'assets': {'thumbna

In [12]:
import requests

collection_url = "https://stac.eodc.eu/ingestion/v1/collections"

# Send a POST request to ingest the STAC Collection
collection_ingest_response = requests.post(collection_url, json=collection.to_dict(), auth=("ingest1", "LTgrc8RYS0TlqUJYyz"))

# Print the response
print("STAC Collection Ingest Response:", collection_ingest_response.text)

STAC Collection Ingest Response: {"id":"clms","description":"The Copernicus Land Monitoring Service (CLMS) is a component of the European Union’s Copernicus Earth observation programme that provides systematic information on land cover, land use, and biophysical parameters across Europe and globally. It delivers consistent, long-term datasets derived primarily from satellite imagery to support environmental monitoring, spatial planning, agriculture, forestry, and climate applications. The service produces several product lines, including pan-European, local, and global components, enabling both high-resolution mapping and large-scale assessments.","stac_version":"1.1.0","links":[{"rel":"items","type":"application/geo+json","href":"https://stac.eodc.eu/ingestion/v1/collections/clms/items"},{"rel":"parent","type":"application/json","href":"https://stac.eodc.eu/ingestion/v1/"},{"rel":"root","type":"application/json","href":"https://stac.eodc.eu/ingestion/v1/"},{"rel":"self","type":"applic

In [ ]:
import requests

items_url = "https://stac.eodc.eu/ingestion/v1/collections/clms/items"

# Send a POST request to ingest the STAC Collection
collection_ingest_response = requests.post(items_url, json=item_vpp.to_dict(), auth=("user", "password"))

print("STAC Collection Ingest Response:", collection_ingest_response.text)



collection_ingest_response = requests.post(items_url, json=item_st.to_dict(), auth=("user", "password"))
# Print the response
print("STAC Collection Ingest Response:", collection_ingest_response.text)

STAC Collection Ingest Response: {"bbox":[8.45,45.6,18.07,49.83],"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[8.45,45.6],[18.07,45.6],[18.07,49.83],[8.45,49.83],[8.45,45.6]]]},"properties":{"title":"Vegetation Phenology and Productivity","description":"HR-VPP product suite provides information on vegetation status and dynamics.","datetime":null,"start_datetime":"2019-01-01T00:00:00Z","end_datetime":"2022-12-31T00:00:00Z"},"id":"clms-vpp","stac_version":"1.1.0","assets":{"data":{"href":"https://data.eodc.eu/collections/CLMS/CLMS.zarr","type":"application/vnd.zarr; version=3","title":"Vegetation Phenology and Productivity","description":"HR-VPP product suite provides information on vegetation status and dynamics.","roles":["data"],"xarray:open_zarr_kwargs":{"group":"VPP","zarr_version":3,"consolidated":true},"bands":[{"name":"AMPL","description":"Season Amplitude"},{"name":"EOSD","description":"End-of-season Date"},{"name":"EOSV","description":"End-of-season Value"},{"n

In [ ]:
# import requests

# vpp_collection_url = "https://dev.stac.eodc.eu/ingestion/v1/collections/clms"
# collection_delete_response = requests.delete(vpp_collection_url, auth=("user", "password"))
# if collection_delete_response.status_code in (200, 204):
#     print("STAC Collection deleted successfully.")
# else:
#     print("Failed to delete STAC Collection:", collection_delete_response.text)


STAC Collection deleted successfully.
